## Getting a random + hillclimber fixed using imports
The goal of this notebook is to get the whole pipeline fixed using as less as code as possible by encapsulating most of the code as reusable code in external .py files

In [1]:
import geopandas as gpd
import shapely
from getpass import getpass

import sys
sys.path.append("../Code")

from helper_functions import *
from loading_data import *

WelkomCorne!


In [2]:
polygons_list = load_geodata_containers()

In [3]:
api_df = load_api_data(prnt=False)

In [ ]:
rel_poi_df = get_db_afvalcluster_info()

In [ ]:
joined = join_api_db(rel_poi_df, api_df)

In [ ]:
joined['rest'], joined['plastic'], joined['papier'], joined['glas'], joined['textiel'], joined['totaal'] = zip(*joined['aantal_per_fractie'].apply(lambda x: containers_per_cluster(x)))

In [ ]:
df_afstandn2 = get_distance_matrix()

In [ ]:
joined_cluster_distance = joined.set_index('s1_afv_nodes').join(df_afstandn2.set_index('van_s1_afv_nodes')).reset_index().rename(columns={'index': 'van_s1_afv_nodes'})

In [ ]:
all_households= create_all_households(rel_poi_df)
good_result = all_households[all_households['uses_container']]

In [ ]:
good_result_rich = add_shortest_distance_to_all_households(all_households, joined_cluster_distance)

In [ ]:
aansluitingen = create_aansluitingen(good_result, joined_cluster_distance)

In [ ]:
avg_distance = calculate_weighted_distance(good_result)
penalties = calculate_enalties(good_result, aansluitingen)

In [ ]:
def total_pipeline():
    polygons_list = load_geodata_containers()
    api_df = load_api_data(prnt=False)
    rel_poi_df = get_db_afvalcluster_info()
    joined = join_api_db(rel_poi_df, api_df)
    joined['rest'], joined['plastic'], joined['papier'], joined['glas'], joined['textiel'], joined['totaal'] = zip(*joined['aantal_per_fractie'].apply(lambda x: containers_per_cluster(x)))
    df_afstandn2 = get_distance_matrix()
    joined_cluster_distance = joined.set_index('s1_afv_nodes').join(df_afstandn2.set_index('van_s1_afv_nodes')).reset_index().rename(columns={'index': 'van_s1_afv_nodes'})
    all_households=create_all_households(joined_cluster_distance)
    good_result = all_households1[all_households1['uses_container']]
    good_result_rich = add_shortest_distance_to_all_households(all_households, joined_cluster_distance)
    aansluitingen = create_aansluitingen(good_result, joined_cluster_distance)
    avg_distance = calculate_weighted_distance(good_result)
    penalties = calculate_enalties(good_result, aansluitingen)
    return joined_cluster_distance, joined, all_households, aansluitingen
    
    

In [ ]:
rel_poi_df